In [1]:
!pip install chromadb langchain langchain-ollama langchain-chroma langchain_community transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 8.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 8.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.2/507.2 kB 10.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 791.7/791.7 kB 9.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface_hub 1.2.1
    Uninstalling huggingface_hub-1.2.1:
      Successfully uninstalled huggingface_hub-1.2.1

[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [4]:
from langchain_classic.chains import RetrievalQA
from langchain_chroma import Chroma
import chromadb
from langchain_ollama import OllamaLLM
from langchain_ollama import OllamaEmbeddings

/home/icc115/.pyenv/versions/3.10.12/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [6]:
# Crear embeddings usando Ollama
# Se asume que Ollama está corriendo en otro servidor y accesible
embeddings = OllamaEmbeddings(model="deepseek-r1:1.5b", base_url="http://localhost:11434")  

In [ ]:

# -------------------------------
# Conexión a Chroma remoto (se ejecuta al iniciar el bot)
# -------------------------------
client = chromadb.HttpClient(host="localhost", port=3000, ssl=False)

vector_store = Chroma(
    collection_name="qa_servicio_internet_manuales",  # tu colección
    client=client,
    embedding_function=embeddings,  # crucial for remote queries!
)

# Inicializa Ollama una sola vez
llm = OllamaLLM(model="llama3.2:1b", temperature=0.7)  # asegúrate que Ollama docker tenga este modelo



In [17]:
# Recuperador desde Chroma
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

# Crear la cadena RAG
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",
    retriever=retriever,
    return_source_documents=False,
)
pregunta = "¿cual es el plan más barato que hay?"
# Generar respuesta
respuesta = qa_chain.invoke(pregunta)

print("Respuesta:", respuesta)

Respuesta: {'query': '¿cual es el plan más barato que hay?', 'result': 'No puedo proporcionar la ayuda que necesitas porque mi modelo de lenguaje aún no ha sido entrenado con una base de datos que incluya la pregunta "¿cual es el plan más barato que se puede usar?"'}
